<a href="https://colab.research.google.com/github/brendanpshea/colab-utilities/blob/main/SQL_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Create a table (if it doesn't already exist)
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)''')

# Insert sample data (this step is only necessary the first time)
cursor.execute('INSERT INTO students (name) VALUES ("Alice"), ("Bob"), ("Charlie")')

# Commit changes and close the connection
conn.commit()
conn.close()


In [7]:
def display_question(question):
    print(f"SQL Question: {question}")


def show_table_schemas():
    # Connect to the SQLite database
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()

    # Retrieve the list of all tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Display the schema for each table
    for table in tables:
        print(f"Table: {table[0]}")
        cursor.execute(f"PRAGMA table_info({table[0]})")
        columns = cursor.fetchall()
        for column in columns:
            print(f"  - {column[1]} ({column[2]})")
        print()  # Print a newline for better readability

    # Close the database connection
    conn.close()


def execute_query_and_allow_retries(correct_query):
    # Connect to the SQLite database
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()

    attempt = 0
    while True:
        attempt += 1
        user_query = input(f"Attempt {attempt}: Please enter your SQL query (or type 'solution' to see the correct answer): ")

        if user_query.lower() == 'solution':
            print("\nCorrect Query:")
            print(correct_query)
            break

        print("\nYour Results:")
        try:
            cursor.execute(user_query)
            user_result = cursor.fetchall()
            for row in user_result:
                print(row)
        except Exception as e:
            print("Error executing your query:", e)
            user_result = None

        print("\nExpected Results:")
        cursor.execute(correct_query)
        correct_result = cursor.fetchall()
        for row in correct_result:
            print(row)

        # Compare results
        if user_result == correct_result:
            print("\nCorrect! Your query produced the expected result.")
            break
        else:
            print("\nIncorrect. Please try again or check the differences in your query results.")

    # Close the database connection
    conn.close()

# Example usage
show_table_schemas()  # Show the table schemas at the beginning

question = "Select all names from the students table."
correct_query = "SELECT name FROM students"

print("\n---\n")
display_question(question)

# The function now includes logic for retries and the option to view the solution
execute_query_and_allow_retries(correct_query)


Table: students
  - id (INTEGER)
  - name (TEXT)


---

SQL Question: Select all names from the students table.
Attempt 1: Please enter your SQL query (or type 'solution' to see the correct answer): SELECT * from students

Your Results:
(1, 'Alice')
(2, 'Bob')
(3, 'Charlie')

Expected Results:
('Alice',)
('Bob',)
('Charlie',)

Incorrect. Please try again or check the differences in your query results.
Attempt 2: Please enter your SQL query (or type 'solution' to see the correct answer): solution

Correct Query:
SELECT name FROM students


In [16]:
import sqlite3
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import Layout
import pandas as pd

# Function to display the table schemas in a nicely formatted HTML table
def show_table_schemas():
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    schema_html = "<h4>Table Schemas</h4>"
    for table in tables:
        schema_html += f"<strong>Table: {table[0]}</strong><br>"
        cursor.execute(f"PRAGMA table_info({table[0]})")
        columns = cursor.fetchall()
        schema_html += "<table><tr><th>Column Name</th><th>Data Type</th></tr>"
        for column in columns:
            schema_html += f"<tr><td>{column[1]}</td><td>{column[2]}</td></tr>"
        schema_html += "</table><br>"
    display(HTML(schema_html))
    conn.close()

def create_query_widget(correct_query):
    text_area = widgets.Textarea(
        value='',
        placeholder='Type your SQL query here...',
        description='Query:',
        layout=Layout(width='60%', height='100px')
    )
    submit_button = widgets.Button(description="Submit")
    retry_button = widgets.Button(description="Retry", layout=Layout(visibility='hidden'))

    def execute_query(b):
        with conn:
            try:
                # Execute user query
                user_result = pd.read_sql_query(text_area.value, conn)
                # Display user results
                display(HTML("<h4>Your Results:</h4>"))
                display(user_result)

                # Execute correct query
                correct_result = pd.read_sql_query(correct_query, conn)
                # Display expected results
                display(HTML("<h4>Expected Results:</h4>"))
                display(correct_result)

                # Compare results
                if user_result.equals(correct_result):
                    display(HTML("<div><strong>Correct!</strong> Your query produced the expected result.</div>"))
                    submit_button.layout.visibility = 'hidden'
                    retry_button.layout.visibility = 'hidden'
                else:
                    display(HTML("<div><strong>Incorrect.</strong> Please try again.</div>"))
                    retry_button.layout.visibility = 'visible'
            except Exception as e:
                display(HTML(f"<div>Error executing your query: {str(e)}</div>"))

    def retry_query(b):
        # Clear previous outputs
        clear_output()
        show_table_schemas()
        display(HTML(question_html))
        display(widget_container)

    submit_button.on_click(execute_query)
    retry_button.on_click(retry_query)

    widget_container = widgets.VBox([text_area, submit_button, retry_button])
    return widget_container

# Display table schemas
show_table_schemas()

# Setup SQLite connection
conn = sqlite3.connect('example.db')

# Display the question
question_html = "<h3>SQL Question:</h3><p>Select all names from the students table.</p>"
display(HTML(question_html))

# Correct query for the example question
correct_query = "SELECT name FROM students"

# Function call to execute the query, allow retries, and display results
query_widget = create_query_widget(correct_query)
display(query_widget)


Column Name,Data Type
id,INTEGER
name,TEXT


,id
0,1
1,2
2,3


,name
0,Alice
1,Bob
2,Charlie


In [36]:
import sqlite3
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import Button, Layout, VBox, Textarea
import pandas as pd

# Function to display the table schemas in a nicely formatted HTML table
def show_table_schemas(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()



    schema_html = "<h2>Database Schema</h2>"
    schema_html += '<table border="1">'
    for table in tables:
        schema_html += f"<h4>Table Name: {table[0]}</h4><br>"
        cursor.execute(f"PRAGMA table_info({table[0]})")
        columns = cursor.fetchall()
        schema_html += "<table><tr><th>Column Name</th><th>Data Type</th></tr>"
        for column in columns:
            schema_html += f"<tr><td>{column[1]}</td><td>{column[2]}</td></tr>"
        schema_html += "</table><br>"
    display(HTML(schema_html))
    conn.close()

# Main function to iterate through questions and process user inputs
def process_questions(db_path, questions, answers):
    if not questions or not answers or len(questions) != len(answers):
        display(HTML("<div>Please provide an equal number of questions and answers.</div>"))
        return

    score = 0

    question_index = 0  # To track the current question

    # Function to display the current question and set up UI for answer submission
    def display_current_question():
        clear_output(wait=True)
        show_table_schemas(db_path)
        question_html = f"<h3>SQL Question {question_index + 1}:</h3><p>{questions[question_index]}</p>"
        display(HTML(question_html))
        text_area.value = ''  # Clear previous input
        display(query_widget)

    # Function to handle query submission
    def submit_query(b):
        with sqlite3.connect(db_path) as conn:
            try:
                user_query = text_area.value
                user_result = pd.read_sql_query(user_query, conn)
                correct_query = answers[question_index]
                correct_result = pd.read_sql_query(correct_query, conn)

                # Display results
                display(HTML("<h4>Your Results:</h4>"))
                display(user_result)
                display(HTML("<h4>Expected Results:</h4>"))
                display(correct_result)
                display(HTML("<h4>Current:</h4>"))
                display(correct_result)

                # Check correctness
                if user_result.equals(correct_result):
                    feedback = "<div style='color: green;'><strong>Correct!</strong></div>"
                    next_button.layout.visibility = 'visible'  # Show the next button
                else:
                    feedback = "<div style='color: red;'><strong>Incorrect.</strong> Try again or proceed to the next question.</div>"
                    retry_button.layout.visibility = 'visible'  # Show the retry button
                display(HTML(feedback))
            except Exception as e:
                display(HTML(f"<div>Error executing your query: {str(e)}</div>"))

    # Function to proceed to the next question
    def next_question(b):
        nonlocal question_index
        question_index += 1
        if question_index < len(questions):
            display_current_question()
        else:
            display(HTML("<div>All questions completed. Well done!</div>"))
            next_button.layout.visibility = 'hidden'
            retry_button.layout.visibility = 'hidden'

    # Function to retry the current question
    def retry_question(b):
        display_current_question()

    text_area = Textarea(value='', placeholder='Type your SQL query here...', description='Query:', layout=Layout(width='60%', height='100px'))
    submit_button = Button(description="Submit")
    next_button = Button(description="Next Question", layout=Layout(visibility='hidden'))
    retry_button = Button(description="Retry", layout=Layout(visibility='hidden'))

    submit_button.on_click(submit_query)
    next_button.on_click(next_question)
    retry_button.on_click(retry_question)

    query_widget = VBox([text_area, submit_button, retry_button, next_button])

    display_current_question()  # Display the first question

# Example usage
db_path = 'example.db'
questions = [
    "Select all names from the students table.",
    "Select count(*) from students where name like 'A%';"
]

answers = [
    "SELECT name FROM students",
    "SELECT count(*) FROM students WHERE name LIKE 'A%'"
]

process_questions(db_path, questions, answers)


Column Name,Data Type
id,INTEGER
name,TEXT
